In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, normaltest
import scipy.stats as stats
import itertools
from scipy.stats import ttest_ind
from scipy.stats import f_oneway

In [11]:
df_holidays = pd.read_csv("C:/Users/palla/Downloads/CorporacionFavorita-Ecuador-Sales-Forecasting-master/CorporacionFavorita-Ecuador-Sales-Forecasting-master/data/holidays_events.csv")

In [12]:
df_holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [13]:
df_holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [14]:
print("Unique Rows:")
print(df_holidays.isnull().sum())
print("Duplicate Rows:", df_holidays.duplicated().sum())

Unique Rows:
date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64
Duplicate Rows: 0


In [15]:
print("Unique values in 'type':", df_holidays['type'].unique())
print("Unique values in 'locale':", df_holidays['locale'].unique())
print("Unique values in 'locale':", df_holidays['locale_name'].unique())
print("Unique values in 'transferred':", df_holidays['transferred'].unique())

Unique values in 'type': ['Holiday' 'Transfer' 'Additional' 'Bridge' 'Work Day' 'Event']
Unique values in 'locale': ['Local' 'Regional' 'National']
Unique values in 'locale': ['Manta' 'Cotopaxi' 'Cuenca' 'Libertad' 'Riobamba' 'Puyo' 'Guaranda'
 'Imbabura' 'Latacunga' 'Machala' 'Santo Domingo' 'El Carmen' 'Cayambe'
 'Esmeraldas' 'Ecuador' 'Ambato' 'Ibarra' 'Quevedo'
 'Santo Domingo de los Tsachilas' 'Santa Elena' 'Quito' 'Loja' 'Salinas'
 'Guayaquil']
Unique values in 'transferred': [False  True]


In [16]:
df_holidays['date'] = pd.to_datetime(df_holidays['date'])
df_holidays['year'] = df_holidays['date'].dt.year
df_holidays['month'] = df_holidays['date'].dt.month
print("Date Range:", df_holidays['date'].min(), "to", df_holidays['date'].max())

Date Range: 2012-03-02 00:00:00 to 2017-12-26 00:00:00


In [17]:
print(df_holidays.describe())

                                date         year       month
count                            350   350.000000  350.000000
mean   2015-04-24 00:45:15.428571392  2014.714286    7.711429
min              2012-03-02 00:00:00  2012.000000    1.000000
25%              2013-12-23 06:00:00  2013.000000    5.000000
50%              2015-06-08 00:00:00  2015.000000    7.000000
75%              2016-07-03 00:00:00  2016.000000   11.000000
max              2017-12-26 00:00:00  2017.000000   12.000000
std                              NaN     1.614253    3.333366


In [18]:
df_holidays.describe(include=['object','bool'])

,type,locale,locale_name,description,transferred
count,350,350,350,350,350
unique,6,3,24,103,2
top,Holiday,National,Ecuador,Carnaval,False
freq,221,174,174,10,338


In [19]:
holiday_counts = df_holidays.groupby(['year']).size()
print(holiday_counts)

year
2012    41
2013    51
2014    67
2015    52
2016    86
2017    53
dtype: int64


In [20]:
# Count of Holidays and Events per year
holiday_counts = df_holidays.groupby(['year','type']).size().unstack()
print(holiday_counts)

type  Additional  Bridge  Event  Holiday  Transfer  Work Day
year                                                        
2012         7.0     2.0    NaN     31.0       1.0       NaN
2013         9.0     NaN    1.0     38.0       1.0       2.0
2014         9.0     1.0   17.0     38.0       1.0       1.0
2015         8.0     1.0    3.0     39.0       NaN       1.0
2016         9.0     1.0   34.0     38.0       3.0       1.0
2017         9.0     NaN    1.0     37.0       6.0       NaN


In [21]:
# Count of Holidays and Events per year
holiday_counts = df_holidays.groupby(['year','locale']).size().unstack()
print(holiday_counts)

locale  Local  National  Regional
year                             
2012       23        14         4
2013       25        22         4
2014       25        38         4
2015       25        23         4
2016       26        56         4
2017       28        21         4


In [22]:
holiday_counts = df_holidays.groupby(['year','type','locale']).size()
print(holiday_counts)

year  type        locale  
2012  Additional  Local        1
                  National     6
      Bridge      National     2
      Holiday     Local       22
                  National     5
                  Regional     4
      Transfer    National     1
2013  Additional  Local        2
                  National     7
      Event       National     1
      Holiday     Local       23
                  National    11
                  Regional     4
      Transfer    National     1
      Work Day    National     2
2014  Additional  Local        2
                  National     7
      Bridge      National     1
      Event       National    17
      Holiday     Local       23
                  National    11
                  Regional     4
      Transfer    National     1
      Work Day    National     1
2015  Additional  Local        1
                  National     7
      Bridge      National     1
      Event       National     3
      Holiday     Local       24
                

In [23]:
holiday_counts = df_holidays.groupby(['year','type','locale','locale_name']).size()
print(holiday_counts)

year  type        locale    locale_name                   
2012  Additional  Local     Quito                             1
                  National  Ecuador                           6
      Bridge      National  Ecuador                           2
      Holiday     Local     Ambato                            2
                            Cayambe                           1
                                                             ..
2017  Holiday     Regional  Santo Domingo de los Tsachilas    1
      Transfer    Local     Cuenca                            1
                            Ibarra                            1
                            Quito                             1
                  National  Ecuador                           3
Length: 180, dtype: int64


In [25]:
holiday_counts = df_holidays.groupby(['year','transferred','locale']).size()
print(holiday_counts)

year  transferred  locale  
2012  False        Local       23
                   National    13
                   Regional     4
      True         National     1
2013  False        Local       25
                   National    21
                   Regional     4
      True         National     1
2014  False        Local       25
                   National    37
                   Regional     4
      True         National     1
2015  False        Local       25
                   National    23
                   Regional     4
2016  False        Local       25
                   National    54
                   Regional     4
      True         Local        1
                   National     2
2017  False        Local       25
                   National    18
                   Regional     4
      True         Local        3
                   National     3
dtype: int64


In [26]:
holiday_counts = df_holidays.groupby(['year','locale','locale_name','transferred']).size()
print(holiday_counts)

year  locale    locale_name                     transferred
2012  Local     Ambato                          False          2
                Cayambe                         False          1
                Cuenca                          False          1
                El Carmen                       False          1
                Esmeraldas                      False          1
                                                              ..
2017  National  Ecuador                         True           3
      Regional  Cotopaxi                        False          1
                Imbabura                        False          1
                Santa Elena                     False          1
                Santo Domingo de los Tsachilas  False          1
Length: 152, dtype: int64


In [27]:
categorical_cols = ['type', 'locale', 'locale_name', 'transferred']
df_holidays['transferred'] = df_holidays['transferred'].astype(str)
chi_square_results = {}

for col1, col2 in itertools.combinations(categorical_cols, 2):
    contingency_table = pd.crosstab(df_holidays[col1], df_holidays[col2])
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    chi_square_results[f"{col1} vs {col2}"] = {
        'Chi-Square Statistic': chi2,
        'P-value': p,
        'Conclusion': "Significant Difference" if p  < 0.05 else "No Significant Difference",
    }

chi_square_df = pd.DataFrame.from_dict(chi_square_results, orient='index')
chi_square_df

,Chi-Square Statistic,P-value,Conclusion
type vs locale,131.458024,2.354491e-23,Significant Difference
type vs locale_name,180.015456,1.019698e-04,Significant Difference
type vs transferred,7.253206,2.024798e-01,No Significant Difference
locale vs locale_name,700.000000,8.785524e-118,Significant Difference
locale vs transferred,1.861984,3.941625e-01,No Significant Difference
locale_name vs transferred,12.380895,9.641658e-01,No Significant Difference


In [28]:
holiday_groups = [df_holidays[df_holidays['type'] == htype]['year'] for htype in df_holidays['type'].unique()]
f_stat, p_value = f_oneway(*holiday_groups)
print(f"ANOVA Test (Year vs Holiday Type) → P-Value: {p_value}")
if p_value < 0.05:
   print("Conclusion: Significant Difference")

ANOVA Test (Year vs Holiday Type) → P-Value: 0.004846774954971729
Conclusion: Significant Difference


In [29]:
df_stores = pd.read_csv("C:/Users/palla/Downloads/CorporacionFavorita-Ecuador-Sales-Forecasting-master/CorporacionFavorita-Ecuador-Sales-Forecasting-master/data/stores.csv")

In [30]:
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [31]:
df_stores.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [32]:
print("Unique Rows:")
print(df_stores.isnull().sum())
print("Duplicate Rows:", df_stores.duplicated().sum())

Unique Rows:
store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64
Duplicate Rows: 0


In [33]:
# Assuming df_stores is the DataFrame containing the stores table data
print("Unique values in 'type':", df_stores['type'].unique())
print("Unique values in 'city':", df_stores['city'].unique())
print("Unique values in 'state':", df_stores['state'].unique())
print("Unique values in 'cluster':", df_stores['cluster'].unique())


Unique values in 'type': ['D' 'B' 'C' 'E' 'A']
Unique values in 'city': ['Quito' 'Santo Domingo' 'Cayambe' 'Latacunga' 'Riobamba' 'Ibarra'
 'Guaranda' 'Puyo' 'Ambato' 'Guayaquil' 'Salinas' 'Daule' 'Babahoyo'
 'Quevedo' 'Playas' 'Libertad' 'Cuenca' 'Loja' 'Machala' 'Esmeraldas'
 'Manta' 'El Carmen']
Unique values in 'state': ['Pichincha' 'Santo Domingo de los Tsachilas' 'Cotopaxi' 'Chimborazo'
 'Imbabura' 'Bolivar' 'Pastaza' 'Tungurahua' 'Guayas' 'Santa Elena'
 'Los Rios' 'Azuay' 'Loja' 'El Oro' 'Esmeraldas' 'Manabi']
Unique values in 'cluster': [13  8  9  4  6 15  7  3 12 16  1 10  2  5 11 14 17]


In [39]:
df_stores['cluster'] = df_stores['cluster'].astype('category')


In [40]:
df_stores.describe(include=['object', 'bool'])


,city,state,type
count,54,54,54
unique,22,16,5
top,Quito,Pichincha,D
freq,18,19,18


In [42]:
df_stores.groupby(['state', 'city'])['type'].unique().reset_index()



,state,city,type
0,Azuay,Cuenca,"[D, B]"
1,Bolivar,Guaranda,[C]
2,Chimborazo,Riobamba,[C]
3,Cotopaxi,Latacunga,[C]
4,El Oro,Machala,"[C, D]"
5,Esmeraldas,Esmeraldas,[E]
6,Guayas,Daule,[D]
7,Guayas,Guayaquil,"[D, E, C, B, A]"
8,Guayas,Libertad,[E]
9,Guayas,Playas,[C]


In [43]:
df_stores.groupby(['state', 'city', 'type']).size().reset_index(name='count')


,state,city,type,count
0,Azuay,Cuenca,B,1
1,Azuay,Cuenca,D,2
2,Bolivar,Guaranda,C,1
3,Chimborazo,Riobamba,C,1
4,Cotopaxi,Latacunga,C,2
5,El Oro,Machala,C,1
6,El Oro,Machala,D,1
7,Esmeraldas,Esmeraldas,E,1
8,Guayas,Daule,D,1
9,Guayas,Guayaquil,A,1


In [45]:
df_stores.groupby(['state', 'city', 'cluster'], observed=True).size().reset_index(name='count')


,state,city,cluster,count
0,Azuay,Cuenca,2,2
1,Azuay,Cuenca,6,1
2,Bolivar,Guaranda,15,1
3,Chimborazo,Riobamba,7,1
4,Cotopaxi,Latacunga,15,2
5,El Oro,Machala,3,1
6,El Oro,Machala,4,1
7,Esmeraldas,Esmeraldas,10,1
8,Guayas,Daule,1,1
9,Guayas,Guayaquil,1,1


In [46]:
df_stores.groupby(['state', 'city', 'type', 'cluster'], observed=True).size().reset_index(name='count')


,state,city,type,cluster,count
0,Azuay,Cuenca,B,6,1
1,Azuay,Cuenca,D,2,2
2,Bolivar,Guaranda,C,15,1
3,Chimborazo,Riobamba,C,7,1
4,Cotopaxi,Latacunga,C,15,2
5,El Oro,Machala,C,3,1
6,El Oro,Machala,D,4,1
7,Esmeraldas,Esmeraldas,E,10,1
8,Guayas,Daule,D,1,1
9,Guayas,Guayaquil,A,17,1


In [51]:
df_type_counts = df_stores.groupby(['state', 'city', 'type']).size().reset_index(name='count')

df_highest_type = df_type_counts.loc[df_type_counts.groupby(['state', 'city'])['count'].idxmax()]
print(df_highest_type)


                             state           city type  count
1                            Azuay         Cuenca    D      2
2                          Bolivar       Guaranda    C      1
3                       Chimborazo       Riobamba    C      1
4                         Cotopaxi      Latacunga    C      2
5                           El Oro        Machala    C      1
7                       Esmeraldas     Esmeraldas    E      1
8                           Guayas          Daule    D      1
11                          Guayas      Guayaquil    C      2
14                          Guayas       Libertad    E      1
15                          Guayas         Playas    C      1
16                        Imbabura         Ibarra    C      1
17                            Loja           Loja    D      1
18                        Los Rios       Babahoyo    B      1
19                        Los Rios        Quevedo    C      1
20                          Manabi      El Carmen    C      1
21      

In [52]:
#which city has highest number of stores
df_stores.groupby(['state', 'city']).size().reset_index(name='count').sort_values(by='count', ascending=False)


,state,city,count
18,Pichincha,Quito,18
7,Guayas,Guayaquil,8
0,Azuay,Cuenca,3
20,Santo Domingo de los Tsachilas,Santo Domingo,3
4,El Oro,Machala,2
3,Cotopaxi,Latacunga,2
21,Tungurahua,Ambato,2
15,Manabi,Manta,2
1,Bolivar,Guaranda,1
2,Chimborazo,Riobamba,1


In [55]:
#Is there an association between the type of store and its cluster?
# Create a contingency table between 'type' and 'cluster'
contingency_table = pd.crosstab(df_stores['type'], df_stores['cluster'])

# Perform Chi-Square Test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Display results
print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p}")

# Interpretation
if p < 0.05:
    print("Conclusion: There is a significant association between store type and cluster.")
else:
    print("Conclusion: There is no significant association between store type and cluster.")


Chi-Square Statistic: 189.87500000000003
P-value: 2.105033183851119e-14
Conclusion: There is a significant association between store type and cluster.


In [58]:
df_oil = pd.read_csv("C:/Users/palla/Downloads/CorporacionFavorita-Ecuador-Sales-Forecasting-master/CorporacionFavorita-Ecuador-Sales-Forecasting-master/data/oil.csv")

In [59]:
# Convert 'date' column to datetime format
df_oil['date'] = pd.to_datetime(df_oil['date'])

# Get the minimum and maximum date
start_date = df_oil['date'].min()
end_date = df_oil['date'].max()

print(f"Date Range: {start_date} to {end_date}")


Date Range: 2013-01-01 00:00:00 to 2017-08-31 00:00:00


In [62]:
# Convert date column to datetime
df_oil['date'] = pd.to_datetime(df_oil['date'])

# Extract year and month
df_oil['year'] = df_oil['date'].dt.year
df_oil['month'] = df_oil['date'].dt.month

# Mean oil price per year
mean_oil_price_per_year = df_oil.groupby('year')['dcoilwtico'].mean().reset_index()

# Mean oil price per month and year
mean_oil_price_per_month_year = df_oil.groupby(['year', 'month'])['dcoilwtico'].mean().reset_index()
print(mean_oil_price_per_year)
print(mean_oil_price_per_month_year)
# Find the highest mean oil price per month-year
highest_avg_oil_price = mean_oil_price_per_month_year.loc[mean_oil_price_per_month_year['dcoilwtico'].idxmax()]

print("Highest Mean Oil Price:")
print(highest_avg_oil_price)


   year  dcoilwtico
0  2013   97.982540
1  2014   93.172222
2  2015   48.656706
3  2016   43.293651
4  2017   49.232874
    year  month  dcoilwtico
0   2013      1   94.756667
1   2013      2   95.308947
2   2013      3   92.938500
3   2013      4   92.021364
4   2013      5   94.509545
5   2013      6   95.772500
6   2013      7  104.670909
7   2013      8  106.572727
8   2013      9  106.289500
9   2013     10  100.538261
10  2013     11   93.864000
11  2013     12   97.625238
12  2014      1   94.617143
13  2014      2  100.817368
14  2014      3  100.803810
15  2014      4  102.069048
16  2014      5  102.177143
17  2014      6  105.794286
18  2014      7  103.588636
19  2014      8   96.536190
20  2014      9   93.211905
21  2014     10   84.396957
22  2014     11   75.789474
23  2014     12   59.290455
24  2015      1   47.219000
25  2015      2   50.584211
26  2015      3   47.823636
27  2015      4   54.452857
28  2015      5   59.265000
29  2015      6   59.819545
30  2015    

In [68]:
df_transactions = pd.read_csv("C:/Users/palla/Downloads/CorporacionFavorita-Ecuador-Sales-Forecasting-master/CorporacionFavorita-Ecuador-Sales-Forecasting-master/data/transactions.csv")

In [69]:
df_transactions['date'] = pd.to_datetime(df_transactions['date'])  # Ensure date is in datetime format

# Get the minimum and maximum date
start_date = df_transactions['date'].min()
end_date = df_transactions['date'].max()

print(f"Date range for transactions table: {start_date} to {end_date}")


Date range for transactions table: 2013-01-01 00:00:00 to 2017-08-15 00:00:00


In [74]:
df_transactions['date'] = pd.to_datetime(df_transactions['date'])  # Ensure date is in datetime format
df_transactions['year'] = df_transactions['date'].dt.year  # Extract year
df_transactions['month'] = df_transactions['date'].dt.month  # Extract month

# Group by year and month, then find the max transactions in each group
highest_transactions = df_transactions.groupby(['year', 'month'])['transactions'].max().reset_index()

# Sort by year, then month in ascending order
highest_transactions = highest_transactions.sort_values(by=['year', 'month'], ascending=False)

print(highest_transactions)


    year  month  transactions
55  2017      8          5037
54  2017      7          5664
53  2017      6          5867
52  2017      5          6398
51  2017      4          5827
50  2017      3          5464
49  2017      2          5418
48  2017      1          5701
47  2016     12          8307
46  2016     11          5690
45  2016     10          5674
44  2016      9          5418
43  2016      8          4830
42  2016      7          5165
41  2016      6          5626
40  2016      5          6194
39  2016      4          5502
38  2016      3          5248
37  2016      2          5153
36  2016      1          5256
35  2015     12          8359
34  2015     11          5772
33  2015     10          5491
32  2015      9          5417
31  2015      8          5370
30  2015      7          5243
29  2015      6          5857
28  2015      5          6839
27  2015      4          5393
26  2015      3          5478
25  2015      2          5303
24  2015      1          5434
23  2014  

In [76]:
df_transactions['date'] = pd.to_datetime(df_transactions['date'])  # Convert date column to datetime
df_transactions['year'] = df_transactions['date'].dt.year  # Extract year
df_transactions['month'] = df_transactions['date'].dt.month  # Extract month

# Group by year and month, then find the max transactions in each group
highest_transactions = df_transactions.groupby(['year', 'month'])['transactions'].max().reset_index()

# Sort by year (ascending) first, then transactions (descending)
highest_transactions = highest_transactions.sort_values(by=['year', 'transactions'], ascending=[True, False])

print(highest_transactions)


    year  month  transactions
11  2013     12          8256
4   2013      5          6328
5   2013      6          5511
9   2013     10          5416
0   2013      1          5401
8   2013      9          5391
7   2013      8          5214
10  2013     11          5203
1   2013      2          5166
2   2013      3          5095
3   2013      4          5062
6   2013      7          4665
23  2014     12          8120
16  2014      5          6280
17  2014      6          6236
20  2014      9          6099
19  2014      8          5945
22  2014     11          5872
21  2014     10          5760
12  2014      1          5600
15  2014      4          5571
14  2014      3          5553
13  2014      2          5233
18  2014      7          5092
35  2015     12          8359
28  2015      5          6839
29  2015      6          5857
34  2015     11          5772
33  2015     10          5491
26  2015      3          5478
24  2015      1          5434
32  2015      9          5417
27  2015  

In [77]:
df_transactions['date'] = pd.to_datetime(df_transactions['date'])  # Convert date column to datetime
df_transactions['year'] = df_transactions['date'].dt.year  # Extract year
df_transactions['month'] = df_transactions['date'].dt.month  # Extract month

# Group by year and month, then find the total transactions per month
monthly_transactions = df_transactions.groupby(['year', 'month'])['transactions'].sum().reset_index()

# Find the month with the highest transactions for each year
highest_transactions_per_year = monthly_transactions.loc[monthly_transactions.groupby('year')['transactions'].idxmax()]

print(highest_transactions_per_year)


    year  month  transactions
11  2013     12       2962866
23  2014     12       3049083
35  2015     12       3149888
47  2016     12       3195971
52  2017      5       2801884


In [78]:
df_train = pd.read_csv("C:/Users/palla/Downloads/CorporacionFavorita-Ecuador-Sales-Forecasting-master/CorporacionFavorita-Ecuador-Sales-Forecasting-master/data/train.csv")

In [79]:
print("Date Range:", df_train["date"].min(), "to", df_train["date"].max())


Date Range: 2013-01-01 to 2017-08-15


In [80]:
import pandas as pd

# Convert the 'date' column to datetime format
df_train['date'] = pd.to_datetime(df_train['date'])

# Extract the year and month from the 'date' column
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month

# Group by year and month and sum the sales for each group
monthly_sales = df_train.groupby(['year', 'month'])['sales'].sum().reset_index()

# Find the month with the highest sales for each year
peak_sales_per_year = monthly_sales.loc[monthly_sales.groupby('year')['sales'].idxmax()]

print(peak_sales_per_year)


    year  month         sales
11  2013     12  1.580312e+07
23  2014     12  2.434045e+07
35  2015     12  2.724398e+07
47  2016     12  2.964029e+07
54  2017      7  2.701148e+07


In [81]:
# Group by 'family' and sum the sales for each product family
sales_by_family = df_train.groupby('family')['sales'].sum().reset_index()

# Sort the values in descending order to find the family with the highest sales
highest_sales_family = sales_by_family.sort_values(by='sales', ascending=False).head(1)

print(highest_sales_family)


       family         sales
12  GROCERY I  3.434627e+08


In [82]:
# Convert 'date' column to datetime format
df_train['date'] = pd.to_datetime(df_train['date'])

# Extract year
df_train['year'] = df_train['date'].dt.year

# Group by 'year' and 'family', summing the sales
sales_by_year_family = df_train.groupby(['year', 'family'])['sales'].sum().reset_index()

# Get the product family with the highest sales for each year
highest_sales_per_year = sales_by_year_family.loc[sales_by_year_family.groupby('year')['sales'].idxmax()]

print(highest_sales_per_year)


     year     family         sales
12   2013  GROCERY I  5.833993e+07
45   2014  GROCERY I  6.566679e+07
78   2015  GROCERY I  7.622854e+07
111  2016  GROCERY I  8.608654e+07
144  2017  GROCERY I  5.714094e+07


In [83]:
# Convert 'date' column to datetime format
df_train['date'] = pd.to_datetime(df_train['date'])

# Extract year and month
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month

# Group by year, month, and family, summing sales
sales_by_month_family = df_train.groupby(['year', 'month', 'family'])['sales'].sum().reset_index()

# Get the product family with the lowest sales for each month in a year
lowest_sales_per_month = sales_by_month_family.loc[sales_by_month_family.groupby(['year', 'month'])['sales'].idxmin()]

# Display result
print(lowest_sales_per_month)


      year  month           family  sales
1     2013      1        BABY CARE    0.0
34    2013      2        BABY CARE    0.0
67    2013      3        BABY CARE    0.0
100   2013      4        BABY CARE    0.0
133   2013      5        BABY CARE    0.0
166   2013      6        BABY CARE    0.0
199   2013      7        BABY CARE    0.0
232   2013      8        BABY CARE    0.0
265   2013      9        BABY CARE    0.0
298   2013     10        BABY CARE    0.0
331   2013     11        BABY CARE    0.0
364   2013     12        BABY CARE    0.0
397   2014      1        BABY CARE    0.0
430   2014      2        BABY CARE    0.0
466   2014      3            BOOKS    0.0
496   2014      4        BABY CARE    0.0
529   2014      5        BABY CARE    0.0
562   2014      6        BABY CARE    0.0
598   2014      7            BOOKS    0.0
628   2014      8        BABY CARE    0.0
664   2014      9            BOOKS    0.0
697   2014     10            BOOKS    0.0
730   2014     11            BOOKS

In [84]:
# Convert 'date' column to datetime format
df_train['date'] = pd.to_datetime(df_train['date'])

# Extract year and month
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month

# Group by year, month, and family, summing the 'onpromotion' column
promotions_by_month = df_train.groupby(['year', 'month', 'family'])['onpromotion'].sum().reset_index()

# Identify product families that had promotions every month in a year
total_months = promotions_by_month.groupby(['year', 'family'])['month'].nunique()
families_with_promotions = total_months[total_months == 12].reset_index()

# Display result
print(families_with_promotions)


     year                      family  month
0    2013                  AUTOMOTIVE     12
1    2013                   BABY CARE     12
2    2013                      BEAUTY     12
3    2013                   BEVERAGES     12
4    2013                       BOOKS     12
..    ...                         ...    ...
127  2016                     POULTRY     12
128  2016              PREPARED FOODS     12
129  2016                     PRODUCE     12
130  2016  SCHOOL AND OFFICE SUPPLIES     12
131  2016                     SEAFOOD     12

[132 rows x 3 columns]


In [85]:
# Convert 'date' to datetime format and extract year
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['year'] = df_train['date'].dt.year

# Group by year, store number, and product family, then sum sales
sales_summary = df_train.groupby(['year', 'store_nbr', 'family'])['sales'].sum().reset_index()

# Display result
print(sales_summary)


      year  store_nbr                      family         sales
0     2013          1                  AUTOMOTIVE     781.00000
1     2013          1                   BABY CARE       0.00000
2     2013          1                      BEAUTY     648.00000
3     2013          1                   BEVERAGES  311948.00000
4     2013          1                       BOOKS       0.00000
...    ...        ...                         ...           ...
8905  2017         54                     POULTRY   13682.04199
8906  2017         54              PREPARED FOODS   17678.00000
8907  2017         54                     PRODUCE  151280.10106
8908  2017         54  SCHOOL AND OFFICE SUPPLIES     610.00000
8909  2017         54                     SEAFOOD     578.00000

[8910 rows x 4 columns]


In [86]:
# Convert 'date' to datetime format and extract year
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['year'] = df_train['date'].dt.year

# Group by year, store number, and product family, then sum sales
sales_summary = df_train.groupby(['year', 'store_nbr', 'family'])['sales'].sum().reset_index()

# Sort sales in descending order
sales_summary = sales_summary.sort_values(by='sales', ascending=False)

# Display result
print(sales_summary)

      year  store_nbr                   family         sales
6810  2016         45                GROCERY I  4.054549e+06
6777  2016         44                GROCERY I  3.897036e+06
6876  2016         47                GROCERY I  3.772779e+06
6795  2016         44                  PRODUCE  3.745801e+06
6768  2016         44                BEVERAGES  3.655321e+06
...    ...        ...                      ...           ...
2542  2014         24                BABY CARE  0.000000e+00
3162  2014         42  PLAYERS AND ELECTRONICS  0.000000e+00
6     2013          1              CELEBRATION  0.000000e+00
84    2013          3                HOME CARE  0.000000e+00
8881  2017         54                    BOOKS  0.000000e+00

[8910 rows x 4 columns]


In [87]:
#Group by year, store number, and product family, then sum sales
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['year'] = df_train['date'].dt.year

promoted_sales = df_train[df_train['onpromotion'] > 0]

# Group by year, store number, and product family, then sum sales
promoted_sales_summary = promoted_sales.groupby(['year', 'store_nbr', 'family'])['sales'].sum().reset_index()

# Sort by sales in descending order
promoted_sales_summary = promoted_sales_summary.sort_values(by='sales', ascending=False)

# Display result
print(promoted_sales_summary)


      year  store_nbr           family        sales
3684  2016         45        GROCERY I  4030213.356
3655  2016         44        GROCERY I  3890270.000
3742  2016         47        GROCERY I  3752416.000
3647  2016         44        BEVERAGES  3620856.000
2207  2015         45        GROCERY I  3551597.935
...    ...        ...              ...          ...
3360  2016         33     PET SUPPLIES        1.000
1126  2015          2        MAGAZINES        1.000
1118  2015          2         HARDWARE        1.000
3130  2016         25         HARDWARE        1.000
5409  2017         54  HOME APPLIANCES        1.000

[5423 rows x 4 columns]
